# Credit Card Detection Training - YOLOv8

This notebook trains a YOLOv8 model for credit card detection with progressive occlusion evaluation.

Features:
- Weights & Biases integration for experiment tracking
- Google Drive integration for saving models
- Progressive occlusion evaluation
- Model size: Medium, 100 epochs


## Step 1: Setup Environment


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Install dependencies
!pip install -q ultralytics roboflow python-dotenv opencv-python matplotlib numpy pandas pyyaml wandb


In [ ]:
# Clone or update your repository
import os
from pathlib import Path

REPO_DIR = "/content/credit_card_yolov12"

if Path(REPO_DIR).exists():
    print("Repository exists, pulling latest changes...")
    %cd {REPO_DIR}
    !git pull origin main
else:
    print("Cloning repository...")
    !git clone https://github.com/Turje/credit_card_yolov12.git
    %cd credit_card_yolov12

# Verify files exist
print("\nVerifying required files...")
required_files = [
    "src/split_dataset.py",
    "src/prepare_progressive_tests.py",
    "src/train.py",
    "src/evaluate_progressive.py"
]

for file in required_files:
    file_path = Path(REPO_DIR) / file
    if file_path.exists():
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - NOT FOUND!")

# Or if you've uploaded files to Drive, uncomment:
# %cd /content/drive/MyDrive/credit_card_yolov12


## Step 2: Setup Weights & Biases


In [ ]:
import wandb

# Login to Weights & Biases
wandb.login(key='6defa0781045a6f791ddd5b18bd7ebbdcdfdc86d')

# Initialize wandb project
wandb.init(
    project="credit-card-detection",
    name="yolov8-medium-100epochs",
    config={
        "model_size": "m",
        "epochs": 100,
        "imgsz": 640,
        "batch": 16,
        "dataset": "credit-cards-coco"
    }
)


### Option A: Download Dataset from Roboflow (Recommended)

**Run this cell to download the dataset directly from Roboflow to Google Drive.**


In [ ]:
# Download dataset from Roboflow
from roboflow import Roboflow
import os
import shutil
from pathlib import Path
import time

# Your Roboflow API key
ROBOFLOW_API_KEY = "uMvNj8tWIWmY9bXNwrf1"

# Initialize Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# Download credit card dataset
project = rf.workspace("efe-efesefe-gvfaz").project("credit-cards-n4hrw")
version = project.version(3)

# Strategy: Download to local /content first (faster), then move to Drive
LOCAL_DOWNLOAD_DIR = "/content/datasets_temp"
DRIVE_DATASET_DIR = "/content/drive/MyDrive/credit_card_yolov12/datasets"

# Create directories
Path(LOCAL_DOWNLOAD_DIR).mkdir(parents=True, exist_ok=True)
Path(DRIVE_DATASET_DIR).mkdir(parents=True, exist_ok=True)

# Change to download directory (Roboflow sometimes downloads to current directory)
original_cwd = os.getcwd()
os.chdir(LOCAL_DOWNLOAD_DIR)

print("Downloading dataset from Roboflow...")
print(f"Current directory: {os.getcwd()}")
print(f"Download location: {LOCAL_DOWNLOAD_DIR}")

# Download in COCO format to local storage first
try:
    print("Starting download...")
    # Try downloading without location first to see where it goes
    dataset = version.download("coco")
    print(f"✅ Download completed!")
    print(f"Roboflow returned location: {dataset.location}")
    
    # Wait a moment for files to be written
    time.sleep(2)
    
    # Check what Roboflow actually created
    print(f"\nChecking download directory contents...")
    download_path = Path(dataset.location) if isinstance(dataset.location, str) else dataset.location
    
    print(f"Roboflow location: {download_path}")
    print(f"Location exists: {download_path.exists()}")
    
    if download_path.exists():
        items = list(download_path.iterdir())
        print(f"Items in {download_path}: {len(items)}")
        for item in items:
            size_info = ""
            if item.is_file():
                try:
                    size_info = f" ({item.stat().st_size} bytes)"
                except:
                    pass
            print(f"  - {item.name} ({'dir' if item.is_dir() else 'file'}){size_info}")
    
    # Also check current directory (Roboflow might download there)
    cwd = Path(os.getcwd())
    if cwd.exists():
        cwd_items = list(cwd.iterdir())
        if cwd_items:
            print(f"\nItems in current directory ({cwd}): {len(cwd_items)}")
            for item in cwd_items[:10]:  # Limit to first 10
                print(f"  - {item.name} ({'dir' if item.is_dir() else 'file'})")
    
    # Check parent directory too
    parent = download_path.parent
    if parent.exists() and parent != download_path:
        parent_items = list(parent.iterdir())
        print(f"\nItems in parent directory ({parent}): {len(parent_items)}")
        for item in parent_items[:10]:
            if item.is_dir() and "credit" in item.name.lower():
                print(f"  📁 {item.name}")
    
except Exception as e:
    print(f"❌ Error during download: {e}")
    import traceback
    traceback.print_exc()
    
    # Try alternative: download to current directory
    print("\nTrying alternative download method...")
    try:
        os.chdir("/content")
        dataset = version.download("coco")
        print(f"Alternative download location: {dataset.location}")
    except Exception as e2:
        print(f"Alternative method also failed: {e2}")
        raise
finally:
    os.chdir(original_cwd)

# Find the actual dataset folder
print(f"\n{'='*60}")
print("Locating downloaded dataset...")

# Roboflow might return a string path or a Path object
if hasattr(dataset, 'location'):
    download_path = Path(dataset.location) if isinstance(dataset.location, str) else dataset.location
else:
    download_path = Path(LOCAL_DOWNLOAD_DIR)

print(f"Roboflow location: {download_path}")
print(f"Checking: {download_path}")
print(f"Exists: {download_path.exists()}")

# Roboflow typically creates a folder named like: credit-cards-n4hrw-3
# Let's search more thoroughly
print(f"\n{'='*60}")
print("Searching for dataset folder...")

# Check multiple possible locations
search_paths = [
    Path(LOCAL_DOWNLOAD_DIR),
    Path("/content"),
    Path("/content/datasets_temp"),
    download_path,
]

# Also check for common Roboflow folder names
possible_names = [
    "credit-cards-n4hrw-3",
    "credit-cards-n4hrw",
    "credit-cards",
    "Credit-Cards-3",
    "Credit-Cards"
]

actual_dataset = None

# First, search in all search paths
for search_path in search_paths:
    if not search_path.exists():
        continue
    
    print(f"\nSearching in: {search_path}")
    
    # Check if train folder exists directly
    if (search_path / "train").exists():
        actual_dataset = search_path
        print(f"  ✅ Found train folder directly!")
        break
    
    # Check subdirectories
    try:
        items = list(search_path.iterdir())
        for item in items:
            if item.is_dir():
                # Check if this folder has train
                if (item / "train").exists():
                    actual_dataset = item
                    print(f"  ✅ Found train folder in: {item.name}")
                    break
                
                # Check nested structure
                try:
                    for subitem in item.iterdir():
                        if subitem.is_dir() and (subitem / "train").exists():
                            actual_dataset = subitem
                            print(f"  ✅ Found train folder in nested: {item.name}/{subitem.name}")
                            break
                except PermissionError:
                    pass
                
                if actual_dataset:
                    break
    except PermissionError:
        print(f"  ⚠️ Permission denied")
        continue
    
    if actual_dataset:
        break

# If still not found, try looking for folders with the expected names
if not actual_dataset:
    print(f"\nTrying specific folder names...")
    for search_path in search_paths:
        if not search_path.exists():
            continue
        for name in possible_names:
            test_path = search_path / name
            if test_path.exists() and test_path.is_dir():
                print(f"  Found folder: {test_path}")
                if (test_path / "train").exists():
                    actual_dataset = test_path
                    print(f"    ✅ Has train folder!")
                    break
                # Check nested
                try:
                    for subitem in test_path.iterdir():
                        if subitem.is_dir() and (subitem / "train").exists():
                            actual_dataset = subitem
                            print(f"    ✅ Found train in nested: {subitem.name}")
                            break
                except:
                    pass
            if actual_dataset:
                break
        if actual_dataset:
            break

# If found, copy to Drive
if actual_dataset:
    source_path = Path(actual_dataset)
    dest_path = Path(DRIVE_DATASET_DIR) / source_path.name
    
    print(f"\n{'='*60}")
    print(f"✅ Dataset found at: {source_path}")
    print(f"Copying to Drive: {dest_path}")
    
    # Copy to Drive
    if dest_path.exists():
        print(f"⚠️ Destination exists, removing old version...")
        shutil.rmtree(dest_path)
    
    shutil.copytree(source_path, dest_path)
    print(f"✅ Dataset copied to Drive!")
    
    ORIGINAL_DATASET = str(dest_path)
    
    # Clean up local temp
    print(f"Cleaning up temporary files...")
    shutil.rmtree(LOCAL_DOWNLOAD_DIR, ignore_errors=True)
    
else:
    print(f"\n❌ Could not find dataset folder with 'train' subdirectory")
    print(f"Please check manually:")
    print(f"  - {download_path}")
    print(f"  - {LOCAL_DOWNLOAD_DIR}")
    print(f"  - /content")
    ORIGINAL_DATASET = str(Path(DRIVE_DATASET_DIR) / "credit-cards-n4hrw-3")

# Verify final dataset
print(f"\n{'='*60}")
print(f"Final dataset path: {ORIGINAL_DATASET}")
print(f"Path exists: {Path(ORIGINAL_DATASET).exists()}")

if Path(ORIGINAL_DATASET).exists():
    train_path = Path(ORIGINAL_DATASET) / "train"
    if train_path.exists():
        train_files = list(train_path.glob("*.json"))
        train_images = list(train_path.glob("*.jpg")) + list(train_path.glob("*.png"))
        print(f"\n✅ Dataset verified!")
        print(f"   - COCO annotations: {len(train_files)}")
        print(f"   - Images: {len(train_images)}")
    else:
        print(f"\n⚠️ Train folder not found at: {train_path}")


### Option B: Use Dataset Already in Drive

**Skip this section if you ran Option A above.**

If you already have the dataset uploaded to Google Drive, skip Option A and make sure it's in:
`/content/drive/MyDrive/credit_card_yolov12/datasets/`


### Diagnostic: Check Dataset Structure

If the dataset wasn't found, run this cell to see what's actually in the download directory:


In [ ]:
# Diagnostic: List all folders and files in the datasets directory
from pathlib import Path

DATASET_BASE = "/content/drive/MyDrive/credit_card_yolov12/datasets"

def list_directory(path, max_depth=3, current_depth=0, prefix=""):
    """Recursively list directory structure."""
    if current_depth > max_depth:
        return
    
    try:
        path_obj = Path(path)
        if not path_obj.exists():
            print(f"{prefix}❌ Path does not exist: {path}")
            return
        
        items = sorted(path_obj.iterdir())
        for i, item in enumerate(items):
            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "
            print(f"{prefix}{current_prefix}{item.name}")
            
            if item.is_dir() and current_depth < max_depth:
                next_prefix = prefix + ("    " if is_last else "│   ")
                list_directory(item, max_depth, current_depth + 1, next_prefix)
    except Exception as e:
        print(f"{prefix}❌ Error: {e}")

print("Dataset directory structure:")
print("=" * 60)
list_directory(DATASET_BASE)


## Step 3: Prepare Dataset


In [ ]:
import sys
import os
from pathlib import Path

# Add src to path
sys.path.insert(0, '/content/credit_card_yolov12')

# Set dataset paths
DATASET_BASE = "/content/drive/MyDrive/credit_card_yolov12/datasets"

# Check if ORIGINAL_DATASET was set in Option A
if 'ORIGINAL_DATASET' not in globals():
    # Find the downloaded dataset folder (Roboflow creates folders like "credit-cards-n4hrw-3")
    dataset_folders = list(Path(DATASET_BASE).glob("credit-cards*"))
    
    # Look for folder with train subdirectory (COCO format)
    actual_dataset = None
    for folder in dataset_folders:
        if folder.is_dir():
            # Check if it has train folder or is the dataset root
            if (folder / "train").exists():
                actual_dataset = folder
                break
            # Also check subdirectories
            for subfolder in folder.iterdir():
                if subfolder.is_dir() and (subfolder / "train").exists():
                    actual_dataset = subfolder
                    break
            if actual_dataset:
                break
    
    if actual_dataset:
        ORIGINAL_DATASET = str(actual_dataset)
        print(f"✅ Found dataset: {ORIGINAL_DATASET}")
    else:
        # Fallback: try common names
        fallback_paths = [
            f"{DATASET_BASE}/credit-cards-n4hrw-3",
            f"{DATASET_BASE}/credit-cards-coco",
            f"{DATASET_BASE}/credit-cards-n4hrw-3/credit-cards-n4hrw-3"
        ]
        for path in fallback_paths:
            if Path(path).exists():
                ORIGINAL_DATASET = path
                print(f"✅ Found dataset (fallback): {ORIGINAL_DATASET}")
                break
        else:
            ORIGINAL_DATASET = fallback_paths[0]
            print(f"⚠️ Using fallback path: {ORIGINAL_DATASET}")

print(f"\nDataset location: {ORIGINAL_DATASET}")
print(f"Dataset exists: {Path(ORIGINAL_DATASET).exists()}")

# List contents to verify
if Path(ORIGINAL_DATASET).exists():
    print(f"\nDataset contents:")
    for item in sorted(Path(ORIGINAL_DATASET).iterdir()):
        item_type = "📁" if item.is_dir() else "📄"
        print(f"  {item_type} {item.name}")
    
    # Check for train folder
    train_path = Path(ORIGINAL_DATASET) / "train"
    if train_path.exists():
        print(f"\n✅ Train folder found!")
        train_files = list(train_path.glob("*.json"))
        train_images = list(train_path.glob("*.jpg")) + list(train_path.glob("*.png"))
        print(f"   - Annotations: {len(train_files)}")
        print(f"   - Images: {len(train_images)}")
    else:
        print(f"\n⚠️ Train folder not found in {ORIGINAL_DATASET}")
        print(f"   Please check the dataset structure")
else:
    print(f"\n❌ Dataset not found at {ORIGINAL_DATASET}")
    print(f"   Available folders in {DATASET_BASE}:")
    if Path(DATASET_BASE).exists():
        for item in Path(DATASET_BASE).iterdir():
            print(f"     - {item.name}")


In [ ]:
# Split dataset
!python src/split_dataset.py --dataset {ORIGINAL_DATASET} --seed 42


In [ ]:
# Generate progressive occlusion test sets
TEST_DATASET = f"{DATASET_BASE}/credit-cards-coco_split/test"
!python src/prepare_progressive_tests.py --test-dataset {TEST_DATASET} --type crop --seed 42


## Step 4: Train Model


In [ ]:
from ultralytics import YOLO
import yaml
import json
from pathlib import Path

# Model configuration
MODEL_SIZE = "m"  # n, s, m, l, x
EPOCHS = 100
IMG_SIZE = 640
BATCH = 16

# Dataset paths
TRAIN_DATASET = f"{DATASET_BASE}/credit-cards-coco_split/train"
VAL_DATASET = f"{DATASET_BASE}/credit-cards-coco_split/val"

print(f"Training on: {TRAIN_DATASET}")
print(f"Validating on: {VAL_DATASET}")


In [ ]:
# Prepare dataset config for YOLOv8
def prepare_yolo_config(train_path, val_path, output_config="dataset.yaml"):
    """Create YOLOv8 dataset config."""
    # Load COCO to get class names
    ann_file = Path(train_path) / "train" / "_annotations.coco.json"
    with open(ann_file, 'r') as f:
        coco_data = json.load(f)
    
    categories = coco_data.get('categories', [])
    class_names = [cat['name'] for cat in sorted(categories, key=lambda x: x['id'])]
    
    # Create YOLOv8 config
    config = {
        'path': str(Path(train_path).parent.absolute()),
        'train': 'train',
        'val': 'val',
        'names': {i: name for i, name in enumerate(class_names)},
        'nc': len(class_names)
    }
    
    with open(output_config, 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Dataset config created: {output_config}")
    print(f"Classes: {class_names}")
    return output_config

config_file = prepare_yolo_config(TRAIN_DATASET, VAL_DATASET)


In [ ]:
# Convert COCO to YOLO format
from src.train import convert_coco_to_yolo

# Convert train set
convert_coco_to_yolo(TRAIN_DATASET)

# Convert val set  
convert_coco_to_yolo(VAL_DATASET)


## Step 5: Save Model to Drive


In [ ]:
# Initialize model
model = YOLO(f"yolov8{MODEL_SIZE}.pt")  # Load pretrained

# Train with wandb integration
results = model.train(
    data=config_file,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,
    project="/content/drive/MyDrive/credit_card_yolov12/models",
    name=f"credit_card_{MODEL_SIZE}",
    exist_ok=True,
    save=True,
    plots=True,
    val=True,
    # Weights & Biases integration
    project_wandb="credit-card-detection",
)


In [ ]:
import shutil

# Paths
MODEL_DIR = results.save_dir
DRIVE_MODEL_DIR = f"/content/drive/MyDrive/credit_card_yolov12/models/credit_card_{MODEL_SIZE}"

# Copy best model to Drive
best_model = Path(MODEL_DIR) / "weights" / "best.pt"
last_model = Path(MODEL_DIR) / "weights" / "last.pt"

Path(DRIVE_MODEL_DIR).mkdir(parents=True, exist_ok=True)

if best_model.exists():
    shutil.copy2(best_model, f"{DRIVE_MODEL_DIR}/best.pt")
    print(f"✅ Best model saved to: {DRIVE_MODEL_DIR}/best.pt")

if last_model.exists():
    shutil.copy2(last_model, f"{DRIVE_MODEL_DIR}/last.pt")
    print(f"✅ Last model saved to: {DRIVE_MODEL_DIR}/last.pt")

# Copy entire training results
shutil.copytree(MODEL_DIR, DRIVE_MODEL_DIR, dirs_exist_ok=True)
print(f"✅ All training results saved to: {DRIVE_MODEL_DIR}")


## Step 6: Evaluate on Progressive Occlusion


In [ ]:
# Evaluate on progressive occlusion test sets
BEST_MODEL = f"{DRIVE_MODEL_DIR}/best.pt"
TEST_SETS_BASE = f"{DATASET_BASE}/credit-cards-coco_split"

!python src/evaluate_progressive.py \
    --model {BEST_MODEL} \
    --test-sets {TEST_SETS_BASE} \
    --output /content/drive/MyDrive/credit_card_yolov12/outputs/progressive_evaluation


## Step 7: Log Results to WandB


In [ ]:
import json
import pandas as pd
from pathlib import Path

# Load progressive evaluation results
results_file = "/content/drive/MyDrive/credit_card_yolov12/outputs/progressive_evaluation/progressive_results.json"

if Path(results_file).exists():
    with open(results_file, 'r') as f:
        eval_results = json.load(f)
    
    # Log to wandb
    for occlusion_level, metrics in eval_results.items():
        wandb.log({
            f"mAP50_occlusion_{occlusion_level}": metrics['mAP50'],
            f"mAP50_95_occlusion_{occlusion_level}": metrics['mAP50_95'],
            f"precision_occlusion_{occlusion_level}": metrics['precision'],
            f"recall_occlusion_{occlusion_level}": metrics['recall'],
            f"f1_occlusion_{occlusion_level}": metrics['f1']
        })
    
    # Log visualization
    plot_file = "/content/drive/MyDrive/credit_card_yolov12/outputs/progressive_evaluation/progressive_occlusion_results.png"
    if Path(plot_file).exists():
        wandb.log({"progressive_occlusion_plot": wandb.Image(plot_file)})
    
    print("✅ Results logged to Weights & Biases")
    print("\nProgressive Occlusion Results:")
    df = pd.DataFrame(eval_results).T
    print(df)
else:
    print("⚠️ Results file not found")


In [ ]:
# Finish wandb run
wandb.finish()
print("✅ Training complete! Check your Weights & Biases dashboard.")
print(f"✅ Model saved to: {DRIVE_MODEL_DIR}")
print(f"✅ Results saved to: /content/drive/MyDrive/credit_card_yolov12/outputs/progressive_evaluation")
